# PASTIS matrix from E-fields

This notebook calculates PASTIS matrices for the low, mid, and high order modes from single-mode E-fields in the focal plane.  
It also calculates matrices on the low order wavefront sensor (LOWFS) and out of band wavefront sensor (OBWFS).

In [ ]:
import os
os.chdir("/Users/asahoo/repos/PASTIS")
import time
from shutil import copy
from astropy.io import fits
import astropy.units as u
import hcipy
import numpy as np
import pastis.util as util    
from pastis.config import CONFIG_PASTIS 
from pastis.e2e_simulators.luvoir_imaging import LuvoirA_APLC 

## Initial setup and parameters

Set up data paths for input and output

In [ ]:
#root_dir = CONFIG_PASTIS.get('local', 'local_data_path')
root_dir = "/Users/asahoo/Desktop/data_repos/multi_mode"
repo_dir = "/Users/asahoo/repos/PASTIS"
coronagraph_design = 'small' # user provides
overall_dir = util.create_data_path(root_dir, telescope='luvoir_'+coronagraph_design)
resDir = os.path.join(overall_dir, 'matrix_numerical')

print(resDir)

Read from configfile how many modes each DM should be able to do

In [ ]:
max_LO = CONFIG_PASTIS.getint('dm_objects', 'number_of_low_order_modes')
max_MID = CONFIG_PASTIS.getint('dm_objects', 'number_of_mid_order_modes')
max_HI = CONFIG_PASTIS.getint('dm_objects', 'number_of_high_order_modes')
num_DM_act = CONFIG_PASTIS.getint('dm_objects', 'number_of_continuous_dm_actuators')

print(f'max_LO: {max_LO}')
print(f'max_MID: {max_MID}')
print(f'max_HI: {max_HI}')
print(f'num_DM_act: {num_DM_act}')

Read some more required parameters from the configfile

In [ ]:
nb_seg = CONFIG_PASTIS.getint('LUVOIR', 'nb_subapertures')
wvln = CONFIG_PASTIS.getfloat('LUVOIR', 'lambda') * 1e-9  # m
diam = CONFIG_PASTIS.getfloat('LUVOIR', 'diameter')  # m
nm_aber = CONFIG_PASTIS.getfloat('LUVOIR', 'calibration_aberration') * 1e-9   # m
sampling = CONFIG_PASTIS.getfloat('LUVOIR', 'sampling')

In [ ]:
print('LUVOIR apodizer design: {}'.format(coronagraph_design)) 
print()
print('Wavelength: {} m'.format(wvln))
print('Telescope diameter: {} m'.format(diam))
print('Number of segments: {}'.format(nb_seg))
print()
print('Sampling: {} px per lambda/D'.format(sampling))

In [ ]:
# Create necessary directories if they don't exist yet
os.makedirs(resDir, exist_ok=True)
os.makedirs(os.path.join(resDir, 'OTE_images'), exist_ok=True)
os.makedirs(os.path.join(resDir, 'psfs'), exist_ok=True)

In [ ]:
#  Copy configfile to resulting matrix directory
util.copy_config(resDir)

In [ ]:
# Create LUVOIR simulator instance
optics_input = os.path.join(util.find_repo_location(), CONFIG_PASTIS.get('LUVOIR', 'optics_path_in_repo'))
luvoir = LuvoirA_APLC(optics_input, coronagraph_design, sampling) 

Generate the deformable mirrors that are required.

**This will take quite some time**

In [ ]:
luvoir.create_global_zernike_mirror(max_LO)
luvoir.create_segmented_mirror(max_MID)
luvoir.create_ripple_mirror(max_HI)
luvoir.create_continuous_deformable_mirror(num_DM_act)

In [ ]:
# Figur out the total number of "actuators" (= effective modes) for each DM
n_LO = luvoir.zernike_mirror.num_actuators 
n_MID = luvoir.sm.num_actuators 
n_HI = luvoir.ripple_mirror.num_actuators 
n_DM = luvoir.dm.num_actuators

print(f'n_LO: {n_LO}')
print(f'n_MID: {n_MID}')
print(f'n_HI: {n_HI}')
print(f'n_DM: {n_DM}')

Define some parameters that are needed for the subsampled LOWFS output.

In [ ]:
z_pup_downsample = CONFIG_PASTIS.getfloat('numerical', 'z_pup_downsample') 
N_pup_z = int(luvoir.pupil_grid.shape[0] / z_pup_downsample)
grid_zernike = hcipy.field.make_pupil_grid(N_pup_z, diameter=luvoir.diam)

## Flatten all DMs and create unaberrated reference PSF

In [ ]:
LO_modes = np.zeros(n_LO)
MID_modes = np.zeros(n_MID)
HI_modes = np.zeros(n_HI)
DM_modes = np.zeros(n_DM)

In [ ]:
luvoir.zernike_mirror.actuators = LO_modes 
luvoir.sm.actuators = MID_modes
luvoir.ripple_mirror.actuators = HI_modes 
luvoir.dm.actuators = DM_modes

In [ ]:
# Calculate the unaberrated coro and direct PSFs in INTENSITY
unaberrated_coro_psf, ref = luvoir.calc_psf(ref=True, display_intermediate=False)

In [ ]:
# Define the normalization factor for contrast units
norm = np.max(ref)

In [ ]:
# Calculate the unaberrated coro and direct PSFs in E-FIELDS
nonaberrated_coro_psf, ref, efield = luvoir.calc_psf(ref=True, display_intermediate=False, return_intermediate='efield')
Efield_ref = nonaberrated_coro_psf.electric_field

## Generate LOW-order PASTIS matrix from E-fields

In [ ]:
print('Generating the E-fields for LOW modes in science plane')
print(f'number of LO modes: {n_LO}')

In [ ]:
# Create lists that will hold measured fields
print(f'Calibration aberration used: {nm_aber} m')
start_time = time.time()
focus_fieldS = []
focus_fieldS_Re = []
focus_fieldS_Im = []

In [ ]:
for pp in range(0, n_LO):
    print(f'Working on mode {pp}/{n_LO}')
    
    # Apply calibration aberration to used mode
    LO_modes = np.zeros(n_LO)
    LO_modes[pp] = nm_aber / 2
    luvoir.zernike_mirror.actuators  = LO_modes
    
    # Calculate coronagraphic E-field and add to lists
    aberrated_coro_psf, inter = luvoir.calc_psf(display_intermediate=False, return_intermediate='efield')
    focus_field1 = aberrated_coro_psf
    focus_fieldS.append(focus_field1)
    focus_fieldS_Re.append(focus_field1.real)
    focus_fieldS_Im.append(focus_field1.imag)

In [ ]:
# Construct the PASTIS matrix from the E-fields
mat_fast = np.zeros([n_LO, n_LO])   # create empty matrix

for i in range(0, n_LO):
    for j in range(0, n_LO):
        test = np.real((focus_fieldS[i].electric_field - Efield_ref) * np.conj(focus_fieldS[j].electric_field - Efield_ref))
        dh_test = (test / norm) * luvoir.dh_mask
        contrast = np.mean(dh_test[np.where(luvoir.dh_mask != 0)])
        mat_fast[i, j] = contrast

In [ ]:
# Normalize by the calibration aberration
matrix_pastis = np.copy(mat_fast)
matrix_pastis /= np.square(nm_aber * 1e9)

Save results out to disk

In [ ]:
filename_matrix = 'PASTISmatrix_num_LO_' + str(max_LO)
hcipy.write_fits(matrix_pastis, os.path.join(resDir, filename_matrix + '.fits'))
print('Matrix saved to:', os.path.join(resDir, filename_matrix + '.fits'))

In [ ]:
filename_matrix = 'EFIELD_Re_matrix_num_LO_' + str(max_LO)
hcipy.write_fits(focus_fieldS_Re, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Real saved to:', os.path.join(resDir, filename_matrix + '.fits'))

In [ ]:
filename_matrix = 'EFIELD_Im_matrix_num_LO_' + str(max_LO)
hcipy.write_fits(focus_fieldS_Im, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Imag saved to:', os.path.join(resDir, filename_matrix + '.fits'))

In [ ]:
end_time = time.time()

In [ ]:
print('Runtime for LO modes:', end_time - start_time, 'sec =', (end_time - start_time) / 60, 'min')
print('Data saved to {}'.format(resDir))

## Generate MID-order PASTIS matrix from E-fields

In [ ]:
print('Generating the Efield for MID modes in science plane')
print(f'number of MID modes: {n_MID}')

In [ ]:
LO_modes = np.zeros(n_LO)
MID_modes = np.zeros(n_MID)
HI_modes = np.zeros(n_HI)

In [ ]:
# Create lists that will hold measured fields
print(f'Calibration aberration used: {nm_aber} m')
start_time = time.time()
focus_fieldS = []
focus_fieldS_Re = []
focus_fieldS_Im = []

In [ ]:
for pp in range(0, n_MID):
    print(f'Working on mode {pp}/{n_MID}')
    
    # Apply calibration aberration to used mode
    MID_modes = np.zeros(n_MID)
    MID_modes[pp] = nm_aber / 2
    luvoir.sm.actuators  = MID_modes
    
    # Calculate coronagraphic E-field and add to lists
    aberrated_coro_psf, inter = luvoir.calc_psf(display_intermediate=False, return_intermediate='efield')
    focus_field1 = aberrated_coro_psf
    focus_fieldS.append(focus_field1)
    focus_fieldS_Re.append(focus_field1.real)
    focus_fieldS_Im.append(focus_field1.imag)

In [ ]:
# Construct the PASTIS matrix from the E-fields
mat_fast = np.zeros([n_MID, n_MID])

for i in range(0, n_MID):
    for j in range(0, n_MID):
        test = np.real((focus_fieldS[i].electric_field - Efield_ref) * np.conj(focus_fieldS[j].electric_field - Efield_ref))
        dh_test = (test / norm) * luvoir.dh_mask
        contrast = np.mean(dh_test[np.where(luvoir.dh_mask != 0)])
        mat_fast[i, j] = contrast

In [ ]:
# Normalize by the calibration aberration
matrix_pastis = np.copy(mat_fast)
matrix_pastis /= np.square(nm_aber * 1e9)  

In [ ]:
filename_matrix = 'PASTISmatrix_num_MID_' + str(max_MID)
hcipy.write_fits(matrix_pastis, os.path.join(resDir, filename_matrix + '.fits'))
print('Matrix saved to:', os.path.join(resDir, filename_matrix + '.fits'))

In [ ]:
filename_matrix = 'EFIELD_Re_matrix_num_MID_' + str(max_MID)
hcipy.write_fits(focus_fieldS_Re, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Real saved to:', os.path.join(resDir, filename_matrix + '.fits'))

In [ ]:
filename_matrix = 'EFIELD_Im_matrix_num_MID_' + str(max_MID)
hcipy.write_fits(focus_fieldS_Im, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Imag saved to:', os.path.join(resDir, filename_matrix + '.fits'))

In [ ]:
end_time = time.time()
print('Runtime for MID modes:', end_time - start_time, 'sec =', (end_time - start_time) / 60, 'min')
print('Data saved to {}'.format(resDir))

## Generate HI-order PASTIS matrix from E-fields

In [ ]:
print('Generating the Efield for HI modes in science plane')
print(f'number of HI modes: {n_HI}')

In [ ]:
LO_modes = np.zeros(n_LO)
MID_modes = np.zeros(n_MID)
HI_modes = np.zeros(n_HI)

In [ ]:
# Create lists that will hold measured fields
print(f'Calibration aberration used: {nm_aber} m')
start_time = time.time()
focus_fieldS = []
focus_fieldS_Re = []
focus_fieldS_Im = []

In [ ]:
for pp in range(0, n_HI):
    print(f'Working on mode {pp}/{n_HI}')
    
    # Apply calibration aberration to used mode
    HI_modes = np.zeros(n_HI)
    HI_modes[pp] = nm_aber / 2
    luvoir.ripple_mirror.actuators = HI_modes
    
    # Calculate coronagraphic E-field and add to lists
    aberrated_coro_psf, inter = luvoir.calc_psf(display_intermediate=False, return_intermediate='efield')
    focus_field1 = aberrated_coro_psf
    focus_fieldS.append(focus_field1)
    focus_fieldS_Re.append(focus_field1.real)
    focus_fieldS_Im.append(focus_field1.imag)

In [ ]:
# Construct the PASTIS matrix from the E-fieldsmat_fast = np.zeros([n_HI, n_HI])
for i in range(0, n_HI):
    for j in range(0, n_HI):
        test = np.real((focus_fieldS[i].electric_field - Efield_ref) * np.conj(focus_fieldS[j].electric_field - Efield_ref))
        dh_test = (test / norm) * luvoir.dh_mask
        contrast = np.mean(dh_test[np.where(luvoir.dh_mask != 0)])
        mat_fast[i, j] = contrast

In [ ]:
# Normalize by the calibration aberration
matrix_pastis = np.copy(mat_fast) 
matrix_pastis /= np.square(nm_aber * 1e9)

In [ ]:
filename_matrix = 'PASTISmatrix_num_HI_' + str(max_HI)
hcipy.write_fits(matrix_pastis, os.path.join(resDir, filename_matrix + '.fits'))
print('Matrix saved to:', os.path.join(resDir, filename_matrix + '.fits'))

In [ ]:
filename_matrix = 'EFIELD_Re_matrix_num_HI_' + str(max_HI)
hcipy.write_fits(focus_fieldS_Re, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Real saved to:', os.path.join(resDir, filename_matrix + '.fits'))

In [ ]:
filename_matrix = 'EFIELD_Im_matrix_num_HI_' + str(max_HI)
hcipy.write_fits(focus_fieldS_Im, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Imag saved to:', os.path.join(resDir, filename_matrix + '.fits'))

In [ ]:
end_time = time.time()
print('Runtime for HI modes:', end_time - start_time, 'sec =', (end_time - start_time) / 60, 'min')
print('Data saved to {}'.format(resDir))

## Generate LOW-order matrix on LOWFS

In [ ]:
print('Generating the Efield for LOW modes through LOWFS')
print('number of LO modes'.format(n_LO))

Flatten DMs

In [ ]:
LO_modes = np.zeros(n_LO)
MID_modes = np.zeros(n_MID)
HI_modes = np.zeros(n_HI)

In [ ]:
luvoir.zernike_mirror.actuators = LO_modes 
luvoir.sm.actuators = MID_modes
luvoir.ripple_mirror.actuators = HI_modes 

In [ ]:
zernike_ref = luvoir.calc_low_order_wfs()

Calculate unaberrated reference E-field on Zernike WFS on a subsampled grid.

In [ ]:
zernike_ref_sub_real = hcipy.field.subsample_field(zernike_ref.real, z_pup_downsample, grid_zernike, statistic='mean')
zernike_ref_sub_imag = hcipy.field.subsample_field(zernike_ref.imag, z_pup_downsample, grid_zernike, statistic='mean')
Efield_ref = zernike_ref_sub_real + 1j*zernike_ref_sub_imag

In [ ]:
# Create lists that will hold measured fields
print(f'Calibration aberration used: {nm_aber} m')
start_time = time.time()
focus_fieldS = []
focus_fieldS_Re = []
focus_fieldS_Im = []

In [ ]:
for pp in range(0, n_LO):
    print(f'Working on mode {pp}/{n_LO}')
    
    # Apply calibration aberration to used mode
    LO_modes = np.zeros(n_LO)
    LO_modes[pp] = nm_aber / 2
    luvoir.zernike_mirror.actuators = LO_modes
    
    # Calculate E-field on Zernike WFS and add to lists
    zernike_meas = luvoir.calc_low_order_wfs()
    zernike_meas_sub_real = hcipy.field.subsample_field(zernike_meas.real, z_pup_downsample, grid_zernike,statistic='mean')
    zernike_meas_sub_imag = hcipy.field.subsample_field(zernike_meas.imag, z_pup_downsample, grid_zernike,statistic='mean')
    focus_field1 = zernike_meas_sub_real + 1j * zernike_meas_sub_imag
    focus_fieldS.append(focus_field1)
    focus_fieldS_Re.append(focus_field1.real)
    focus_fieldS_Im.append(focus_field1.imag)

In [ ]:
filename_matrix = 'EFIELD_LOWFS_Re_matrix_num_LO_' + str(max_LO)
hcipy.write_fits(focus_fieldS_Re, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Real saved to:', os.path.join(resDir, filename_matrix + '.fits'))

In [ ]:
filename_matrix = 'EFIELD_LOWFS_Im_matrix_num_LO_' + str(max_LO)
hcipy.write_fits(focus_fieldS_Im, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Imag saved to:', os.path.join(resDir, filename_matrix + '.fits'))

In [ ]:
end_time = time.time()
print('Runtime for LO modes and LOWFS:', end_time - start_time, 'sec =', (end_time - start_time) / 60, 'min')
print('Data saved to {}'.format(resDir))

## Generate MID-order matrix on OBWFS

In [ ]:
print('Generating the Efield for MID modes to OBWFS')
print('number of MID modes'.format(n_MID))

In [ ]:
# Flatten DMs
LO_modes = np.zeros(n_LO)
MID_modes = np.zeros(n_MID)
HI_modes = np.zeros(n_HI)

luvoir.zernike_mirror.actuators = LO_modes
luvoir.sm.actuators = MID_modes
luvoir.ripple_mirror.actuators = HI_modes

In [ ]:
# Calculate unaberrated reference E-field on Zernike WFS on a subsampled grid.
zernike_ref = luvoir.calc_out_of_band_wfs()
zernike_ref_sub_real = hcipy.field.subsample_field(zernike_ref.real, z_pup_downsample, grid_zernike, statistic='mean')
zernike_ref_sub_imag = hcipy.field.subsample_field(zernike_ref.imag, z_pup_downsample, grid_zernike, statistic='mean')
Efield_ref = zernike_ref_sub_real + 1j*zernike_ref_sub_imag

In [ ]:
# Create lists that will hold measured fields
print(f'Calibration aberration used: {nm_aber} m')
start_time = time.time()
focus_fieldS = []
focus_fieldS_Re = []
focus_fieldS_Im = []

In [ ]:
for pp in range(0, n_MID):
    print(f'Working on mode {pp}/{n_MID}')
    
    # Apply calibration aberration to used mode
    MID_modes = np.zeros(n_MID)
    MID_modes[pp] = nm_aber / 2
    luvoir.sm.actuators = MID_modes
    
    # Calculate E-field on OBWFS and add to lists
    zernike_meas = luvoir.calc_out_of_band_wfs()
    zernike_meas_sub_real = hcipy.field.subsample_field(zernike_meas.real, z_pup_downsample, grid_zernike, statistic='mean')
    zernike_meas_sub_imag = hcipy.field.subsample_field(zernike_meas.imag, z_pup_downsample, grid_zernike, statistic='mean')
    focus_field1 = zernike_meas_sub_real + 1j * zernike_meas_sub_imag
    focus_fieldS.append(focus_field1)
    focus_fieldS_Re.append(focus_field1.real)
    focus_fieldS_Im.append(focus_field1.imag)

In [ ]:
filename_matrix = 'EFIELD_OBWFS_Re_matrix_num_MID_' + str(max_MID)
hcipy.write_fits(focus_fieldS_Re, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Real saved to:', os.path.join(resDir, filename_matrix + '.fits'))

In [ ]:
filename_matrix = 'EFIELD_OBWFS_Im_matrix_num_MID_' + str(max_MID)
hcipy.write_fits(focus_fieldS_Im, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Imag saved to:', os.path.join(resDir, filename_matrix + '.fits'))

In [ ]:
end_time = time.time()
print('Runtime for MID modes and OBWFS:', end_time - start_time, 'sec =', (end_time - start_time) / 60, 'min')
print('Data saved to {}'.format(resDir))

# Generate HI spatial frequency matrix

In [ ]:
linear model corresponds to e2e model.